In [1]:
import numpy as np

In [192]:
from attr import define,field, attrib

In [3]:
data = open('../inputs/input-day22').read().splitlines()

In [4]:
init = data[:20]

In [5]:
def parse(line):
    order, line = line.split(' ')
    ranges = []
    for obj in line.split(','):
        ranges.append(list(map(lambda x: int(x) + 50,obj[2:].split('..'))))
    return (order, ranges)

In [6]:
init = [parse(l) for l in init]

In [7]:
arr = np.zeros((105,105,105))

In [8]:
for line in init:
    num = 0
    order,(x,y,z) = line
    if order == 'on':
        num = 1
    arr[x[0]:x[1]+1,y[0]:y[1]+1,z[0]:z[1]+1] = num

In [9]:
(arr == 1).sum()

653798

--------
# Task 2

In [1040]:
@define(frozen=True)
class Range():
    beg : int = field()
    end : int = field()
    
    @end.validator
    def _check_end(self, attribute, value):
        if value < self.beg:
             raise ValueError("beg not <= end")
    
    def __repr__(self):
        return "R({},{})".format(self.beg, self.end)
    
    def size(self):
        return self.end - self.beg + 1
    
    def intersect(self, ran2):
        if (self.end < ran2.beg or
            ran2.end < self.beg):
            return None
        beg = max(self.beg, ran2.beg)
        end = min(self.end, ran2.end)
        if (beg <= end):
            return Range(beg, end)
    
    def split(self, range2):
        "The 2nd item is the intersected one, so we can later drop it if we want"
        try:
            if self.beg == range2.beg == range2.end == self.end:
                ans = [Range(self.beg, self.beg)]

            elif self.beg == range2.beg:
                end1 = self.end if self.end < range2.end else range2.end
                end2 = range2.end if end1 == self.end else self.end
                ans = [Range(end1+1, end2), Range(self.beg, end1), ]

            elif self.end == range2.end:
                beg1 = self.beg if self.beg < range2.beg else range2.beg
                beg2 = range2.beg if beg1 == self.beg else self.beg
                ans = [Range(beg1, beg2 - 1), Range(beg2, self.end)]

            else:
                beg1 = self.beg if self.beg < range2.beg else range2.beg
                beg2 = range2.beg if beg1 == self.beg else self.beg
                end1 = self.end if self.end < range2.end else range2.end
                end2 = range2.end if end1 == self.end else self.end
                ans = [Range(beg1, beg2-1), Range(beg2, end1), Range(end1 + 1, end2)]
        except:
            return [Range(self.beg, self.beg)]
        return ans

In [1041]:
assert Range(1,5).intersect(Range(2,6)) == Range(2,5)
assert Range(1,5).intersect(Range(-2,3)) == Range(1,3)
assert Range(1,5).intersect(Range(3,4)) == Range(3,4)
assert Range(1,5).intersect(Range(0,6)) == Range(1,5)

The 2nd item is always the intersect one, we can use it to treat is as subtraction. 

In [1042]:
Range(1,5).split(Range(1,3))

[R(4,5), R(1,3)]

In [1043]:
assert Range(1,5).split(Range(2,5)) == [Range(1,1), Range(2,5)]
assert Range(1,5).split(Range(1,3)) == [Range(4,5), Range(1,3)]
assert Range(1,5).split(Range(3,4)) == [Range(1,2), Range(3,4), Range(5,5)]

In [1055]:
@define(frozen=True)
class Cube():
    x : Range
    y : Range
    z : Range
    
    def __repr__(self):
        return "C({}, {}, {})".format(self.x, self.y,self.z)
    
    def size(self):
        return (self.x.size() * 
                self.y.size() * 
                self.z.size())
    
    def intersect(self, cube):
        x = self.x.intersect(cube.x)
        y = self.y.intersect(cube.y)
        z = self.z.intersect(cube.z)
        if x and y and z:
            return Cube(x,y,z)

    def diff(self, cube, subtract=True):
        "cube is the intersection cube"
        cubes = set()
        x_range = self.x.split(cube.x)
        y_range = self.y.split(cube.y)
        z_range = self.z.split(cube.z)
        for i,x in enumerate(x_range):
            for j,y in enumerate(y_range):
                for k,z in enumerate(z_range):
                    if i == j == k == 1:
                        continue
                    cubes.add(Cube(x,y,z))
        return cubes

Simple intersection

In [1056]:
c1 = Cube(Range(5,5),Range(3,10),Range(4,10))

In [1057]:
c2 = Cube(Range(3,10),Range(6,12),Range(-4,8))

In [1058]:
print(c1.size(), c2.size())

56 728


In [1059]:
diff = c1.intersect(c2)
diff

C(R(5,5), R(6,10), R(4,8))

In [1060]:
c1.diff(diff)

{C(R(5,5), R(3,5), R(4,8)),
 C(R(5,5), R(3,5), R(9,10)),
 C(R(5,5), R(6,10), R(4,8)),
 C(R(5,5), R(6,10), R(9,10))}

In [1061]:
sum([l.size() for l in c1.diff(diff)])

56

In [1062]:
sum(l.size() for l in c1.diff(diff,subtract=True))

56

More complicated intersections

In [1063]:
c1 = Cube(Range(0,2),Range(0,2),Range(0,2))

In [1064]:
c2 = Cube(Range(1,3),Range(1,3),Range(1,3))

In [1065]:
print(c1.size(), c2.size())

27 27


In [1066]:
diff = c1.intersect(c2)

In [1067]:
diff = c2.intersect(c1)

In [1068]:
c1.diff(diff)

{C(R(0,0), R(0,0), R(0,0)),
 C(R(0,0), R(0,0), R(1,2)),
 C(R(0,0), R(1,2), R(0,0)),
 C(R(0,0), R(1,2), R(1,2)),
 C(R(1,2), R(0,0), R(0,0)),
 C(R(1,2), R(0,0), R(1,2)),
 C(R(1,2), R(1,2), R(0,0))}

In [1069]:
c2.diff(diff)

{C(R(1,2), R(1,2), R(3,3)),
 C(R(1,2), R(3,3), R(1,2)),
 C(R(1,2), R(3,3), R(3,3)),
 C(R(3,3), R(1,2), R(1,2)),
 C(R(3,3), R(1,2), R(3,3)),
 C(R(3,3), R(3,3), R(1,2)),
 C(R(3,3), R(3,3), R(3,3))}

In [1070]:
a = list(c2.diff(diff))
a

[C(R(3,3), R(1,2), R(3,3)),
 C(R(3,3), R(1,2), R(1,2)),
 C(R(3,3), R(3,3), R(3,3)),
 C(R(1,2), R(1,2), R(3,3)),
 C(R(3,3), R(3,3), R(1,2)),
 C(R(1,2), R(3,3), R(1,2)),
 C(R(1,2), R(3,3), R(3,3))]

In [1071]:
for i in a:
    for j in a:
        if i != j:
            print(i.intersect(j))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [1027]:
sum([l.size() for l in c1.diff(diff)])

19

In [1028]:
sum(l.size() for l in c1.diff(diff,subtract=True))

19

### Sanity check: the first task using the Cube class calculations

In [1029]:
data = data[:20]

In [1030]:
data = """on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10""".splitlines()

In [1072]:
cubes = set()

In [1073]:
for line in data[:2]:
    print(line)
    order, line = line.split(' ')
    order = order == 'on'
    ranges = [list(map(lambda x: int(x),obj[2:].split('..'))) for obj in line.split(',')]
    cube = Cube(Range(*ranges[0]), Range(*ranges[1]), Range(*ranges[2]))
    cube_parts = set([cube])
    if order:
        new_cubes = set()
        while len(cube_parts) > 0:
            cube = cube_parts.pop()
            for c in cubes:
                intersection = cube.intersect(c)
                if intersection:
                    if intersection == cube:
                        break
                    else:
                        diff = cube.diff(intersection)
                        cube_parts |= diff
                        break
            new_cubes.add(cube)
        cubes |= new_cubes
    else:
        new_cubes = set()
        for c in cubes:
            intersection = c.intersect(cube)
            if intersection and (intersection != cube):
                diff = c.diff(intersection, subtract=True)
                new_cubes |= diff
            else:
                new_cubes.add(c)
        cubes = new_cubes
    print(sum(l.size() for l in cubes))

on x=10..12,y=10..12,z=10..12
27
on x=11..13,y=11..13,z=11..13
73


In [1074]:
a = list(cubes)

In [1075]:
for i in a:
    for j in a:
        if i != j:
            inter = i.intersect(j)
            if inter:
                print(i,j,i.intersect(j))

C(R(11,12), R(13,13), R(13,13)) C(R(11,13), R(11,13), R(11,13)) C(R(11,12), R(13,13), R(13,13))
C(R(11,12), R(13,13), R(11,12)) C(R(11,13), R(11,13), R(11,13)) C(R(11,12), R(13,13), R(11,12))
C(R(11,13), R(11,13), R(11,13)) C(R(11,12), R(13,13), R(13,13)) C(R(11,12), R(13,13), R(13,13))
C(R(11,13), R(11,13), R(11,13)) C(R(11,12), R(13,13), R(11,12)) C(R(11,12), R(13,13), R(11,12))
C(R(11,13), R(11,13), R(11,13)) C(R(10,12), R(10,12), R(10,12)) C(R(11,12), R(11,12), R(11,12))
C(R(11,13), R(11,13), R(11,13)) C(R(13,13), R(13,13), R(13,13)) C(R(13,13), R(13,13), R(13,13))
C(R(11,13), R(11,13), R(11,13)) C(R(13,13), R(13,13), R(11,12)) C(R(13,13), R(13,13), R(11,12))
C(R(11,13), R(11,13), R(11,13)) C(R(11,12), R(11,12), R(13,13)) C(R(11,12), R(11,12), R(13,13))
C(R(11,13), R(11,13), R(11,13)) C(R(13,13), R(11,12), R(13,13)) C(R(13,13), R(11,12), R(13,13))
C(R(11,13), R(11,13), R(11,13)) C(R(13,13), R(11,12), R(11,12)) C(R(13,13), R(11,12), R(11,12))
C(R(10,12), R(10,12), R(10,12)) C(R(11,1

In [1076]:
cube = Cube(Range(11,13), Range(11,13), Range(11,13))

In [1077]:
for c in cubes:
    intersection = cube.intersect(c)
    if intersection:
        diff = cube.diff(intersection, subtract=True)
        print(diff)
        cube_parts |= set(diff)
        break
cubes.add(cube)

{C(R(11,12), R(13,13), R(11,12)), C(R(13,13), R(13,13), R(13,13)), C(R(13,13), R(13,13), R(11,12)), C(R(11,12), R(11,12), R(13,13)), C(R(13,13), R(11,12), R(13,13)), C(R(11,12), R(11,12), R(11,12)), C(R(13,13), R(11,12), R(11,12))}


In [461]:
cube

Cube(x=Range(beg=11, end=13), y=Range(beg=11, end=13), z=Range(beg=11, end=13))

In [462]:
cube = cube_parts.pop()

In [463]:
cube_parts

{Cube(x=Range(beg=11, end=12), y=Range(beg=13, end=13), z=Range(beg=11, end=12)),
 Cube(x=Range(beg=11, end=12), y=Range(beg=13, end=13), z=Range(beg=13, end=13)),
 Cube(x=Range(beg=13, end=13), y=Range(beg=11, end=12), z=Range(beg=11, end=12)),
 Cube(x=Range(beg=13, end=13), y=Range(beg=11, end=12), z=Range(beg=13, end=13)),
 Cube(x=Range(beg=13, end=13), y=Range(beg=13, end=13), z=Range(beg=11, end=12)),
 Cube(x=Range(beg=13, end=13), y=Range(beg=13, end=13), z=Range(beg=13, end=13))}

In [400]:
cubes

{Cube(x=Range(beg=10, end=12), y=Range(beg=10, end=12), z=Range(beg=10, end=12)),
 Cube(x=Range(beg=11, end=13), y=Range(beg=11, end=13), z=Range(beg=11, end=13))}

In [333]:
sum([l.size() for l in cube_parts])

19

In [284]:
sum(c.size() for c in cubes)

65

In [285]:
cubes

{Cube(x=Range(beg=10, end=10), y=Range(beg=10, end=10), z=Range(beg=10, end=10)),
 Cube(x=Range(beg=10, end=11), y=Range(beg=10, end=11), z=Range(beg=12, end=12)),
 Cube(x=Range(beg=10, end=11), y=Range(beg=12, end=12), z=Range(beg=10, end=11)),
 Cube(x=Range(beg=10, end=11), y=Range(beg=12, end=12), z=Range(beg=12, end=12)),
 Cube(x=Range(beg=11, end=11), y=Range(beg=11, end=11), z=Range(beg=12, end=13)),
 Cube(x=Range(beg=11, end=11), y=Range(beg=12, end=13), z=Range(beg=11, end=11)),
 Cube(x=Range(beg=11, end=11), y=Range(beg=12, end=13), z=Range(beg=12, end=13)),
 Cube(x=Range(beg=11, end=12), y=Range(beg=11, end=12), z=Range(beg=13, end=13)),
 Cube(x=Range(beg=11, end=12), y=Range(beg=13, end=13), z=Range(beg=11, end=12)),
 Cube(x=Range(beg=11, end=12), y=Range(beg=13, end=13), z=Range(beg=13, end=13)),
 Cube(x=Range(beg=12, end=12), y=Range(beg=10, end=11), z=Range(beg=10, end=11)),
 Cube(x=Range(beg=12, end=12), y=Range(beg=10, end=11), z=Range(beg=12, end=12)),
 Cube(x=Range(be

In [274]:
cubes

{Cube(x=Range(beg=-11, end=37), y=Range(beg=-44, end=1), z=Range(beg=-20, end=25))}

In [240]:
stack

set()

In [239]:
data[:3]

['on x=-9..45,y=-15..37,z=-2..46',
 'on x=-48..2,y=-11..37,z=3..48',
 'on x=-47..4,y=-47..-3,z=-12..34']

In [112]:
diff = Range(2,5).intersect(Range(3,6))

In [114]:
Range(2,5).split(diff)

[Range(beg=2, end=3), Range(beg=4, end=5)]

In [9]:
def parse(line):
    order, line = line.split(' ')
    ranges = []
    for obj in line.split(','):
        ranges.append(list(map(lambda x: int(x),obj[2:].split('..'))))
    return (order, ranges)

In [10]:
data = [parse(l) for l in data]

In [12]:
data[:10]

[('on', [[-9, 45], [-15, 37], [-2, 46]]),
 ('on', [[-48, 2], [-11, 37], [3, 48]]),
 ('on', [[-47, 4], [-47, -3], [-12, 34]]),
 ('on', [[-24, 29], [-12, 40], [-20, 29]]),
 ('on', [[-48, 3], [-38, 16], [-22, 31]]),
 ('on', [[-35, 13], [-10, 43], [-11, 42]]),
 ('on', [[-4, 47], [1, 47], [-26, 18]]),
 ('on', [[-49, 1], [-29, 22], [-44, 2]]),
 ('on', [[-12, 33], [-14, 38], [-9, 35]]),
 ('on', [[-44, 8], [0, 46], [-28, 20]])]

In [17]:
a = [3,]

In [18]:
b = a.pop()

In [ ]:
def union(objs, cube):
    stack = [cube]
    while len(stack) > 0:
        cube = stack.pop()
        for obj in objs:
            if cube[]

In [82]:
def merge(state, size, line):
    """
    State is of the form [list,list,list]
    of turned-on ranges in each axis
    """
    x, y, z = state
    action, (l_x,l_y,l_z) = line
    
    if action == 'on':
        x_diff = set(np.arange(l_x[0],l_x[1]+1)) - x
        y_diff = set(np.arange(l_y[0],l_y[1]+1)) - y
        z_diff = set(np.arange(l_z[0],l_z[1]+1)) - z
        
        size += len(x_diff) * len(x_diff) * len(x_diff)
        
        x |= x_diff
        y |= y_diff
        z |= z_diff

    elif action == 'off':
        x_int = x & set(np.arange(l_x[0],l_x[1]+1))
        y_int = y & set(np.arange(l_y[0],l_y[1]+1))
        z_int = z & set(np.arange(l_z[0],l_z[1]+1))
        
        size -= len(x_int) * len(y_int) * len(z_int)

        x -= x_int
        y -= y_int
        z -= z_int

    return [x,y,z], size

In [83]:
state = [set(), set(), set()]
size = 0
for line in data[:20]:
    state, size = merge(state, size, line)

In [86]:
get_size(state)

635040

In [80]:
def get_size(state):
    x,y,z = state
    return len(x) * len(y) * len(z)

In [21]:
a = data[0][1][0]

In [26]:
a

[-9, 45]

In [103]:
from scipy.sparse import dok_matrix, bsr_matrix

In [14]:
import sparse

In [76]:
data = open('../inputs/input-day22').read().splitlines()

In [78]:
data = [parse(l) for l in data]

In [93]:
arr = sparse.DOK(shape = (400000,400000,400000), dtype=int, fill_value=0)

In [ ]:
for line in data:
    num = 0
    order,(x,y,z) = line
    if order == 'on':
        num = 1
    arr[x[0]:x[1]+1,y[0]:y[1]+1,z[0]:z[1]+1] = num

In [85]:
arr = sparse.COO(arr)

In [86]:
arr.sum()

653798

In [71]:
arr

Format,coo
Data Type,bool
Shape,"(200000, 200000, 200000)"
nnz,0
Density,0.0
Read-only,True
Size,0
Storage ratio,0.0


In [49]:
arr[5] = 1

TypeError: 'COO' object does not support item assignment

In [44]:
(arr == 1)

/usr/local/lib/python3.9/site-packages/sparse/_utils.py:323: RuntimeWarning: invalid value encountered in double_scalars
  density = np.float_(arr.nnz) / np.float_(arr.size)
/usr/local/lib/python3.9/site-packages/sparse/_utils.py:343: RuntimeWarning: invalid value encountered in double_scalars
  np.float_(arr.nbytes)


Format,coo
Data Type,bool
Shape,"(0, 0, 0)"
nnz,0
Density,nan
Read-only,True
Size,0
Storage ratio,nan
